In [1]:
import sys 
sys.path.append('../scripts/')
from robot import *
from scipy.stats import multivariate_normal
import random #追加
import copy

In [2]:
class Particle:  ###mclglobal1particle
    def __init__(self, init_pose, weight):
        if init_pose is None:
            self.pose = np.array([
                np.random.uniform(-5.0, 5.0),
                np.random.uniform(-5.0, 5.0),
                np.random.uniform(-math.pi, math.pi)]
            ).T
        else:
            self.pose = init_pose
            
        self.weight = weight
        
    def motion_update(self, nu, omega, time, noise_rate_pdf): 
        ns = noise_rate_pdf.rvs()
        pnu = nu + ns[0]*math.sqrt(abs(nu)/time) + ns[1]*math.sqrt(abs(omega)/time)
        pomega = omega + ns[2]*math.sqrt(abs(nu)/time) + ns[3]*math.sqrt(abs(omega)/time)
        self.pose = IdealRobot.state_transition(pnu, pomega, time, self.pose)
        
    def observation_update(self, observation, envmap, distance_dev_rate, direction_dev):  #変更
        for d in observation:
            obs_pos = d[0]
            obs_id = d[1]
            
            ##パーティクルの位置と地図からランドマークの距離と方角を算出##
            pos_on_map = envmap.landmarks[obs_id].pos
            particle_suggest_pos = IdealCamera.relative_polar_pos(self.pose, pos_on_map)
            
            ##尤度の計算##
            distance_dev = distance_dev_rate*particle_suggest_pos[0]
            cov = np.diag(np.array([distance_dev**2, direction_dev**2]))
            self.weight *= multivariate_normal(mean=particle_suggest_pos, cov=cov).pdf(obs_pos)

In [3]:
class Mcl:  
    def __init__(self, envmap, init_pose, num, motion_noise_stds, \
                 distance_dev_rate=0.14, direction_dev=0.05):
        self.particles = [Particle(init_pose, 1.0/num) for i in range(num)]
        self.map = envmap
        self.distance_dev_rate = distance_dev_rate
        self.direction_dev = direction_dev

        v = motion_noise_stds
        c = np.diag([v["nn"]**2, v["no"]**2, v["on"]**2, v["oo"]**2])
        self.motion_noise_rate_pdf = multivariate_normal(cov=c)
        
        self.ml = self.particles[0] #追加
        
    def set_ml(self): #追加
        i = np.argmax([p.weight for p in self.particles])
        self.ml = self.particles[i]
        
    def motion_update(self, nu, omega, time): 
        for p in self.particles: p.motion_update(nu, omega, time, self.motion_noise_rate_pdf)
            
    def observation_update(self, observation): 
        for p in self.particles:
            p.observation_update(observation, self.map, self.distance_dev_rate, self.direction_dev) 
        self.set_ml() #リサンプリング前に実行
        self.resampling() 
            
    def resampling(self): 
        ws = [e.weight for e in self.particles]    # 重みのリストを作る
        
        #重みの和がゼロに丸め込まれるとサンプリングできなくなるので小さな数を足しておく
        if sum(ws) < 1e-100: ws = [e + 1e-100 for e in ws]
        
        # パーティクルのリストから、weightsのリストの重みに比例した確率で、num個選ぶ    
        ps = random.choices(self.particles, weights=ws, k=len(self.particles))  
        
        # 選んだリストからパーティクルを取り出し、重みを均一に
        self.particles = [copy.deepcopy(e) for e in ps]
        for p in self.particles: p.weight = 1.0/len(self.particles)
        
    def draw(self, ax, elems):  
        xs = [p.pose[0] for p in self.particles]
        ys = [p.pose[1] for p in self.particles]
        vxs = [math.cos(p.pose[2])*p.weight*len(self.particles) for p in self.particles] #重みを要素に反映
        vys = [math.sin(p.pose[2])*p.weight*len(self.particles) for p in self.particles]  #重みを要素に反映
        elems.append(ax.quiver(xs, ys, vxs, vys, \
                               angles='xy', scale_units='xy', scale=1.5, color="blue", alpha=0.5)) #変更

In [4]:
class MclAgent(Agent): 
    def __init__(self, time_interval, nu, omega, particle_pose, envmap, particle_num=100, \
                motion_noise_stds={"nn":0.19, "no":0.001, "on":0.13, "oo":0.2}): #2行目にenvmapを追加
        super().__init__(nu, omega)
        self.pf = Mcl(envmap, particle_pose, particle_num, motion_noise_stds) #envmapを追加
        self.time_interval = time_interval
        
        self.prev_nu = 0.0
        self.prev_omega = 0.0
        
    def decision(self, observation=None): 
        self.pf.motion_update(self.prev_nu, self.prev_omega, self.time_interval)
        self.prev_nu, self.prev_omega = self.nu, self.omega
        self.pf.observation_update(observation)
        return self.nu, self.omega
        
    def draw(self, ax, elems):
        self.pf.draw(ax, elems)
        x, y, t = self.pf.ml.pose #以下追加
        s = "({:.2f}, {:.2f}, {})".format(x,y,int(t*180/math.pi)%360)
        elems.append(ax.text(x, y+0.1, s, fontsize=8))

In [5]:
def trial(animation): ###mclglobal1test（下のTrialのところまで）
    time_interval = 0.1
    world = World(30, time_interval, debug=not animation) 

    ## 地図を生成して3つランドマークを追加 ##
    m = Map()                                  
    m.append_landmark(Landmark(-4,2))
    m.append_landmark(Landmark(2,-3))
    m.append_landmark(Landmark(3,3))
    world.append(m)

    ## ロボットを作る ##
    init_pose = np.array([
        np.random.uniform(-5.0, 5.0),
        np.random.uniform(-5.0, 5.0),
        np.random.uniform(-math.pi, math.pi)]
    ).T
        
    a = MclAgent(time_interval, 0.2, 10.0/180*math.pi, None, m, particle_num=1000) #初期姿勢をNoneに
    r = Robot(init_pose, sensor=Camera(m), agent=a, color="red")
    world.append(r)

    world.draw()
    
    return (r.pose, a.pf.ml.pose)

In [6]:
ok = 0 ###mclglobal1exec
for i in range(1000):
    actual, estm = trial(False)
    diff = math.sqrt((actual[0]-estm[0])**2 + (actual[1]-estm[1])**2)
    print(i, "真値:", actual, "推定値", estm, "誤差:", diff)
    if diff <= 1.0:
        ok += 1

ok

0 真値: [-0.67514941 -1.26875449  3.48829796] 推定値 [-0.94544399 -0.52532226  3.45302152] 誤差: 0.7910440146801706
1 真値: [-3.57763851  0.24996245  4.74974339] 推定値 [-3.56516307  0.33194552  4.8714481 ] 誤差: 0.08292683057193678
2 真値: [3.10659426 0.64388035 8.25578634] 推定値 [3.42629778 0.72766686 8.46746319] 誤差: 0.33050040753270626
3 真値: [-2.71437244 -1.85790825  2.42503199] 推定値 [ 5.54787829 -0.51309557  6.34786794] 誤差: 8.370980130474978
4 真値: [ 1.02803016 -4.15063446  7.08253547] 推定値 [ 0.76935077 -4.66471173  7.14243598] 誤差: 0.5754915033150763
5 真値: [-0.22822847  0.32571678  7.6685034 ] 推定値 [-0.01551825  0.60268488  7.50128478] 誤差: 0.3492233786735353
6 真値: [-3.45002146 -3.96223542  6.16455612] 推定値 [-4.25749366 -4.37490354  6.20661629] 誤差: 0.9068110777772216
7 真値: [1.29874497 1.61021681 8.89640673] 推定値 [1.31928604 1.83602441 8.8779942 ] 誤差: 0.22673994828504498
8 真値: [-4.5434412  -2.75232962  7.67202316] 推定値 [-0.95697651  5.00353817  3.81691302] 誤差: 8.544952557485297
9 真値: [5.07775599 3.10552983 3

/usr/local/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


20 真値: [-1.78533325  3.66536224  5.27012516] 推定値 [-1.42207526  0.5359321   4.4792342 ] 誤差: 3.150442730756891
21 真値: [-2.89494176  4.65917034  9.13843201] 推定値 [-4.90840239 -0.57322278  5.73298512] 誤差: 5.606421447823484
22 真値: [0.14113493 2.05437169 6.90881422] 推定値 [0.14845223 1.9724889  7.03966983] 誤差: 0.08220908144003304
23 真値: [-4.64343738  0.92005904  3.95006671] 推定値 [-3.67388722  0.52390813  4.92891791] 誤差: 1.0473600401421361
24 真値: [ 1.31546232 -1.70138312  6.0820865 ] 推定値 [ 1.93068852 -1.58012627  5.61225433] 誤差: 0.6270618058515409
25 真値: [-6.15182108 -2.9721005   6.01065868] 推定値 [-0.70488677 -2.46981732  7.19594031] 誤差: 5.470044044919101
26 真値: [ 5.72354525 -0.20710631  2.75861666] 推定値 [-2.22916752 -0.31402259  4.43330719] 誤差: 7.9534314241602635
27 真値: [ 6.51904949 -1.7845831   2.86774577] 推定値 [1.15484179 3.69047053 4.37811956] 誤差: 7.664915948555396
28 真値: [2.47917852 1.88705689 6.31433389] 推定値 [-0.03680559 -4.98207237  5.47959438] 誤差: 7.315402437327627
29 真値: [0.52490051 5.04167

96 真値: [ 3.2155851  -4.94182491  7.45706511] 推定値 [ 1.14516825 -5.25175562  6.42377631] 誤差: 2.093485850999245
97 真値: [ 3.62417296 -4.57063764  8.67681438] 推定値 [ 4.3927813  -3.50303738  2.78228775] 誤差: 1.315495761760679
98 真値: [ 3.18569192 -1.77916088  5.37793654] 推定値 [-2.18248332  3.11522376  3.34147254] 誤差: 7.264454990193439
99 真値: [-0.03720645  3.30621182  8.48138436] 推定値 [0.14103259 3.0837207  8.01980926] 誤差: 0.28508148982736853
100 真値: [-0.93253643  3.09975615  1.08878149] 推定値 [-5.11806716 -1.54296295  5.07421476] 誤差: 6.250880596320677
101 真値: [3.69257113 2.04064823 7.16120615] 推定値 [ 0.16012215 -4.88710745  6.10536525] 誤差: 7.776374124282226
102 真値: [0.98967688 2.36840205 1.65012584] 推定値 [0.80809921 2.38339666 1.61103087] 誤差: 0.18219574143399167
103 真値: [-3.65554809  1.46660446  2.82415786] 推定値 [-3.56891522  1.59236129  3.05474085] 誤差: 0.15270898414489212
104 真値: [-2.65535104  5.80427275  4.74824304] 推定値 [-2.6629906   5.9186311   4.80229492] 誤差: 0.11461323436337115
105 真値: [ 2.127859

171 真値: [2.26034712 0.67187558 4.20302706] 推定値 [2.32658535 0.64743518 4.26329604] 誤差: 0.07060337426234771
172 真値: [1.42280129 3.66888099 2.63177439] 推定値 [0.96135537 3.94951426 2.69073908] 誤差: 0.5400808913847597
173 真値: [ 4.97929599 -2.33480007  1.81607798] 推定値 [-2.48059213  1.68871768  6.44010812] 誤差: 8.475766978705405
174 真値: [-0.21971555  3.92212794  2.16174227] 推定値 [-0.42722143  3.22714252  8.76722228] 誤差: 0.7253022965071031
175 真値: [0.99508334 0.71061054 4.31169587] 推定値 [ 3.88424043 -4.25660383  3.3442862 ] 誤差: 5.74634207877757
176 真値: [-0.67416739 -5.46548258  2.32877041] 推定値 [0.89737272 0.21776558 6.57332494] 誤差: 5.896528467433773
177 真値: [-1.5673727   3.90396515  5.22194743] 推定値 [-2.57515647 -0.23807956  4.39518223] 誤差: 4.262881953791172
178 真値: [-5.37405929 -2.1844264   4.22189866] 推定値 [-5.33711455 -2.23961417  4.51736616] 誤差: 0.06641237424642796
179 真値: [ 4.55225276 -3.24829698  4.2935764 ] 推定値 [-1.5938699  -4.52242626  7.79341074] 誤差: 6.27680087701085
180 真値: [-3.32816533  4.

...

698 真値: [5.37042796 0.62299021 2.85594266] 推定値 [ 1.47028977 -0.62259714  1.75591971] 誤差: 4.094211246163849
699 真値: [ 0.31036145 -0.78922187  4.05521586] 推定値 [ 0.80986803 -5.98674817  6.27249528] 誤差: 5.221473592849207
700 真値: [-1.85486137 -1.35412886  5.69221288] 推定値 [-1.48919822 -0.87957213  5.58557367] 誤差: 0.5990939988666067
701 真値: [ 0.25723851 -2.69618376  5.10934514] 推定値 [ 0.15751683 -2.6108443   5.10487423] 誤差: 0.13125257205006233
702 真値: [-2.03064693  0.06909757  3.75379198] 推定値 [0.43377217 0.31985611 4.37135938] 誤差: 2.477143791289427
703 真値: [-2.71498469 -2.42201969  3.21120359] 推定値 [3.48272495 1.89809116 7.47807002] 誤差: 7.554797319909289
704 真値: [2.86847463 0.55219658 5.55226757] 推定値 [2.6734383  0.17410737 5.62135624] 誤差: 0.4254299225999244
705 真値: [0.53261103 3.00238927 5.61087832] 推定値 [0.29947985 3.00658784 5.53958776] 誤差: 0.23316898449143192
706 真値: [4.39886601 0.48801517 3.83725155] 推定値 [5.24708265 0.19052265 3.7012584 ] 誤差: 0.8988733385716929
707 真値: [-5.01535026 -3.374877

773 真値: [ 5.49482944 -2.25513306  2.42437299] 推定値 [2.4626948  0.34502087 3.62989371] 誤差: 3.9943260913553678
774 真値: [3.10408211 2.35843414 3.11009717] 推定値 [2.88100702 2.37173717 2.92885701] 誤差: 0.22347139583525688
775 真値: [2.17104738 2.94140126 3.04847377] 推定値 [1.82868153 3.02876476 3.00375805] 誤差: 0.3533366035852616
776 真値: [-2.60597735 -4.36650488  7.87127205] 推定値 [-1.31849292 -4.68821685  7.90026693] 誤差: 1.3270699943433504
777 真値: [-2.72795809  1.42311787  6.49217872] 推定値 [-2.57481806  0.92232094  6.50535656] 誤差: 0.523688295293021
778 真値: [ 4.01923318 -3.74357884  4.68041413] 推定値 [ 0.74692202 -1.99127507  7.39250884] 誤差: 3.7119521631306323
779 真値: [-3.29966071 -2.71922556  7.91228727] 推定値 [-4.31807383 -2.68580478  7.67186655] 誤差: 1.018961349088579
780 真値: [ 1.49174748 -1.3508547   2.47449436] 推定値 [ 1.08809487 -1.7691702   2.29904262] 誤差: 0.5813116975114876
781 真値: [-4.17747196  2.43715169  4.90141647] 推定値 [-4.44356229  2.38307684  5.20872034] 誤差: 0.2715292816810886
782 真値: [-2.98586

849 真値: [2.96754802 2.44888333 1.76693037] 推定値 [2.97263815 2.36996057 1.76755145] 誤差: 0.07908673693532445
850 真値: [4.58533425 0.041144   3.82501712] 推定値 [ 4.09212002 -0.12653037  3.8283839 ] 誤差: 0.5209366295986921
851 真値: [-4.76629372 -6.97586957  6.55917094] 推定値 [-0.66288097 -2.04037349  2.70956333] 誤差: 6.418498096726187
852 真値: [-3.8390023   0.93018692  3.72624617] 推定値 [-4.08904355  0.90360604  3.78432696] 誤差: 0.2514501279884681
853 真値: [-1.3609227   3.93106153  5.66340335] 推定値 [-1.23500827  3.83923261  5.67192744] 誤差: 0.1558428544897516
854 真値: [-1.09980531 -0.23742395  3.4572392 ] 推定値 [-0.60373766 -0.509312    3.37635695] 誤差: 0.5656909239498813
855 真値: [-3.48045562  3.73454019  7.85753257] 推定値 [-2.80144071  0.71664797  5.76470999] 誤差: 3.0933371431308547
856 真値: [-3.42494746 -0.15470751  5.85909204] 推定値 [-2.10797603  2.82961478  7.21880719] 誤差: 3.261992233152386
857 真値: [-0.12650405 -2.09725861  6.4806803 ] 推定値 [-0.22892506 -1.98046079  6.44582587] 誤差: 0.15534411696653902
858 真値: [2

925 真値: [-1.79192174  4.01949716  2.76754095] 推定値 [-6.54047135  1.20712435  5.36488996] 誤差: 5.518891579815333
926 真値: [-1.01595547  2.67541738  6.67916487] 推定値 [-2.74870898  2.85715987  6.61370626] 誤差: 1.7422586133187228
927 真値: [-2.1834118   0.97392397  7.88390533] 推定値 [-2.53450777  0.49464132  7.6321501 ] 誤差: 0.5941213964621457
928 真値: [-3.18193515  3.83456073  8.66384104] 推定値 [-2.70506747  2.15576762  8.33142679] 誤差: 1.745207467283349
929 真値: [-0.87488611 -0.96284588  4.12098057] 推定値 [-1.16891365 -0.19885099  4.25857079] 誤差: 0.8186210330931498
930 真値: [ 5.56557166 -1.73535742  8.4562482 ] 推定値 [ 5.85433247 -2.78022631  8.42930496] 誤差: 1.0840358862481745
931 真値: [5.97284935e-03 2.27563825e+00 6.90842189e+00] 推定値 [-0.08874175  2.50125287  6.80646744] 誤差: 0.24468921441425673
932 真値: [ 1.41563676 -2.70271793  8.00083238] 推定値 [ 1.50594918 -2.51130383  8.04845728] 誤差: 0.2116499220831115
933 真値: [-0.0595434  -3.44825323  7.8383884 ] 推定値 [-0.19162244 -3.51293104  7.93392979] 誤差: 0.1470649194

459